# Tasks 1 & 3: Data Extraction + Transform and Load

In [3]:
!pip install pyarrow

  Using cached pyarrow-21.0.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (3.3 kB)
Using cached pyarrow-21.0.0-cp311-cp311-macosx_12_0_arm64.whl (31.2 MB)


In [1]:
import pandas as pd

df = pd.DataFrame({"a": [1, 2, 3]})
df.to_parquet("test.parquet", index=False)
pd.read_parquet("test.parquet")

,a
0,1
1,2
2,3


In [1]:
from pathlib import Path
import os
import zipfile
import sqlite3
import pandas as pd

# PATHS
RAW_DIR = Path("data/raw")
PROCESSED = Path("data/processed")
DB_FILE = "supermarket.db"

# 1. DOWNLOAD DATA FROM KAGGLE
def download_from_kaggle():
    """
    Downloads the dataset from Kaggle using the API and extracts it.
    Requires kaggle.json to be configured in ~/.kaggle
    """
    RAW_DIR.mkdir(parents=True, exist_ok=True)
    print("Downloading dataset...")
    os.system("kaggle datasets download -d lovishbansal123/sales-of-a-supermarket -p data/raw")

    zip_path = next(RAW_DIR.glob("*.zip"), None)
    if not zip_path:
        raise FileNotFoundError("ZIP not found. Download may have failed.")
    
    with zipfile.ZipFile(zip_path) as zf:
        zf.extractall(RAW_DIR)
    print(f"Downloaded and extracted: {zip_path.name}")

# 2. TRANSFORM TO STAR SCHEMA
def build_dimensions(df: pd.DataFrame):
    # Rename relevant columns up front
    df = df.rename(columns={
        "Product line": "product_line",
        "Unit price": "unit_price",
        "Branch": "branch",
        "City": "city"
    })

    # Product dimension
    dim_product = (
        df[["product_line", "unit_price"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    dim_product["product_id"] = dim_product.index + 1

    # Store dimension
    dim_store = (
        df[["branch", "city"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    dim_store["store_id"] = dim_store.index + 1

    return dim_product, dim_store, df  # also return renamed df for use in fact

def build_fact(df, dim_product, dim_store):
    fact = (
        df.merge(dim_product, on=["product_line", "unit_price"])
          .merge(dim_store, on=["branch", "city"])
          .assign(sale_id=lambda d: d.index + 1)
          .rename(columns={
              "Date": "date",
              "Customer type": "customer_type",
              "Gender": "gender",
              "Total": "total",
              "Quantity": "quantity",
              "Payment": "payment"
          })
    )

    fact_cols = ["sale_id", "date", "customer_type", "gender",
                 "product_id", "store_id", "total", "quantity", "payment"]
    return fact[fact_cols]


# 3. LOAD TO SQLITE + SAVE TO PARQUET
def write_to_sqlite_and_parquet(dim_product, dim_store, fact_sales):
    # Save to SQLite
    conn = sqlite3.connect(DB_FILE)
    dim_product.to_sql("dim_product", conn, index=False, if_exists="replace")
    dim_store.to_sql("dim_store", conn, index=False, if_exists="replace")
    fact_sales.to_sql("fact_sales", conn, index=False, if_exists="replace")
    conn.close()
    print(f"SQLite DB created: {DB_FILE}")

    # Save to Parquet
    PROCESSED.mkdir(parents=True, exist_ok=True)
    dim_product.to_parquet(PROCESSED / "dim_product.parquet", index=False)
    dim_store.to_parquet(PROCESSED / "dim_store.parquet", index=False)
    fact_sales.to_parquet(PROCESSED / "fact_sales.parquet", index=False)
    print(f"Parquet files written to: {PROCESSED.resolve()}")

# 4. HELPER FUNCTION TO QUERY DB

def run_query(q):
    """
    Executes a SQL query on the SQLite DB and returns a DataFrame.
    """
    with sqlite3.connect(DB_FILE) as conn:
        return pd.read_sql(q, conn)


def show_tables():
    """
    calls the run_query() function to return a list of all tables and views in the database.
    """
    q = "SELECT name, type FROM sqlite_master WHERE type IN ('table', 'view')"
    return run_query(q)


# 5. EXECUTE PIPELINE

if __name__ == "__main__":
    download_from_kaggle()

    # Load raw CSV (extracted filename may vary slightly)
    csv_file = next(RAW_DIR.glob("*.csv"))
    df = pd.read_csv(csv_file)

    # Transform
    dim_product, dim_store, df_renamed = build_dimensions(df)
    fact_sales = build_fact(df_renamed, dim_product, dim_store)

    # Load
    write_to_sqlite_and_parquet(dim_product, dim_store, fact_sales)

Dataset URL: https://www.kaggle.com/datasets/lovishbansal123/sales-of-a-supermarket
License(s): apache-2.0

Downloaded and extracted: sales-of-a-supermarket.zip
SQLite DB created: supermarket.db
Parquet files written to: /Users/faridun/66degrees-supermarket-pipeline/notebooks/data/processed


100%|██████████| 35.9k/35.9k [00:00<00:00, 65.1MB/s]


# Task 2: Schema Design

## Show tables in DB

In [2]:
show_tables()

,name,type
0,dim_product,table
1,dim_store,table
2,fact_sales,table


## Table schema

In [4]:
tables = ["dim_product", "dim_store", "fact_sales"]

for table in tables:
    print(f"Schema for table: {table}")
    display(run_query(f"PRAGMA table_info('{table}')"))

Schema for table: dim_product


,cid,name,type,notnull,dflt_value,pk
0,0,product_line,TEXT,0,None,0
1,1,unit_price,REAL,0,None,0
2,2,product_id,INTEGER,0,None,0


Schema for table: dim_store


,cid,name,type,notnull,dflt_value,pk
0,0,branch,TEXT,0,None,0
1,1,city,TEXT,0,None,0
2,2,store_id,INTEGER,0,None,0


Schema for table: fact_sales


,cid,name,type,notnull,dflt_value,pk
0,0,sale_id,INTEGER,0,None,0
1,1,date,TEXT,0,None,0
2,2,customer_type,TEXT,0,None,0
3,3,gender,TEXT,0,None,0
4,4,product_id,INTEGER,0,None,0
5,5,store_id,INTEGER,0,None,0
6,6,total,REAL,0,None,0
7,7,quantity,INTEGER,0,None,0
8,8,payment,TEXT,0,None,0


## Table Count

In [3]:
q = """
SELECT 'dim_product' AS table_name, COUNT(*) AS row_count FROM dim_product
UNION
SELECT 'dim_store', COUNT(*) FROM dim_store
UNION
SELECT 'fact_sales', COUNT(*) FROM fact_sales;
"""
run_query(q)

,table_name,row_count
0,dim_product,993
1,dim_store,3
2,fact_sales,1000


# Task 4: Reporting

## Rank product line by sales per day

In [5]:
q= """
SELECT 
  s.date,
  p.product_line,
  SUM(s.total) AS total_sales,
  RANK() OVER (PARTITION BY s.date ORDER BY SUM(s.total) DESC) AS rank_by_day
FROM fact_sales s
JOIN dim_product p ON s.product_id = p.product_id
GROUP BY s.date, p.product_line
ORDER BY s.date, total_sales DESC;"""
run_query(q)

,date,product_line,total_sales,rank_by_day
0,1/1/2019,Electronic accessories,1382.7450,1
1,1/1/2019,Sports and travel,1143.3870,2
2,1/1/2019,Food and beverages,1066.0230,3
3,1/1/2019,Fashion accessories,621.2430,4
4,1/1/2019,Home and lifestyle,399.7560,5
...,...,...,...,...
449,3/9/2019,Sports and travel,1877.9565,2
450,3/9/2019,Food and beverages,1459.1115,3
451,3/9/2019,Electronic accessories,980.0070,4
452,3/9/2019,Health and beauty,762.5520,5


## Save query results as a dataset

In [8]:
query = """
SELECT 
  s.date,
  p.product_line,
  SUM(s.total) AS total_sales,
  RANK() OVER (PARTITION BY s.date ORDER BY SUM(s.total) DESC) AS rank_by_day
FROM fact_sales s
JOIN dim_product p ON s.product_id = p.product_id
GROUP BY s.date, p.product_line
ORDER BY s.date, total_sales DESC;"""

report_df = run_query(query)

PROCESSED = Path("data/processed")
PROCESSED.mkdir(parents=True, exist_ok=True)

report_df.to_parquet(PROCESSED / "daily_product_sales_report.parquet", index=False)

## Sales by product line

In [6]:
run_query("""
SELECT 
  p.product_line,
  COUNT(*) AS num_sales,
  ROUND(SUM(f.total), 2) AS total_revenue
FROM fact_sales f
JOIN dim_product p ON f.product_id = p.product_id
GROUP BY p.product_line
ORDER BY total_revenue DESC;
""")

,product_line,num_sales,total_revenue
0,Food and beverages,174,56144.84
1,Sports and travel,166,55122.83
2,Electronic accessories,170,54337.53
3,Fashion accessories,178,54305.90
4,Home and lifestyle,160,53861.91
5,Health and beauty,152,49193.74


In [7]:
run_query("""
SELECT 
  SUM(CASE WHEN customer_type IS NULL THEN 1 ELSE 0 END) AS null_customer_type,
  SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS null_gender
FROM fact_sales;
""")

,null_customer_type,null_gender
0,0,0
